In [ ]:
# Create function to load dataset from csv file. Data from month column will be output as X_month,
# data from day column will be output as X_day, data from hour column will be output as X_hour,
# data from tx_count will be output as y, and all other data will be output as X_numeric.
# All data will be converted to tensors.

import os
import pandas as pd
import torch

def load_dataset(
    csv_file: os.PathLike,
    window_size: int
) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
    # Read the raw data from the file
    df = pd.read_csv(csv_file)

    # Conver the data to tensors
    raw_X_month = torch.tensor(df['month'].values)
    raw_X_day = torch.tensor(df['day'].values)
    raw_X_hour = torch.tensor(df['hour'].values)
    raw_X_numeric = torch.tensor(df.drop(['month', 'day', 'hour', 'y'], axis=1).values).float()
    raw_y = torch.tensor(df['y'].values).float()

    # Make sure we are using a window size that makes sense with our data
    window_size = min(window_size, len(raw_X_month))

    # Get sliding windows of the data
    X_month = raw_X_month.unfold(0, window_size, 1)
    X_day = raw_X_day.unfold(0, window_size, 1)
    X_hour = raw_X_hour.unfold(0, window_size, 1)
    X_numeric = raw_X_numeric.unfold(0, window_size, 1)
    y = raw_y.unfold(0, window_size, 1)

    return X_month, X_day, X_hour, X_numeric, y

In [ ]:
# Create a PyTorch model that takes in 4 inputs (X_month, X_day, X_hour, X_numeric)
# and outputs a series of predictions (y). It will feed X_month, X_day, and X_hour into embedding layers,
# which will then be concatenated with X_numeric and fed into an LSTM layer.

import torch.nn as nn

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.embedding_month = nn.Embedding(12, 5)
        self.embedding_day = nn.Embedding(31, 5)
        self.embedding_hour = nn.Embedding(24, 5)
        self.lstm = nn.LSTM(19, 5, bidirectional=True, batch_first=True)
        self.linear = nn.Linear(10, 1)

    def forward(self, X_month, X_day, X_hour, X_numeric):
        X_month = self.embedding_month(X_month)
        X_day = self.embedding_day(X_day)
        X_hour = self.embedding_hour(X_hour)
        X = torch.cat((X_month, X_day, X_hour, X_numeric), dim=1)
        X, _ = self.lstm(X)
        X = self.linear(X)
        return X

In [ ]:
# Create a function to train an instance of Model on a given dataset.

import torch.optim as optim
from tqdm import tqdm

def train_model(
    model: Model,
    X_month: torch.Tensor,
    X_day: torch.Tensor,
    X_hour: torch.Tensor,
    X_numeric: torch.Tensor,
    y: torch.Tensor,
    epochs: int
):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    pbar = tqdm(range(epochs))
    for _ in pbar:
        model.train()
        optimizer.zero_grad()

        y_pred = model(X_month, X_day, X_hour, X_numeric)
        loss = criterion(y_pred, y)

        loss.backward()
        optimizer.step()

        pbar.set_description(f'Loss: {loss.item()}')

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

def cross_validate(X_month, X_day, X_hour, X_numeric, y, cv=5):
    tscv = TimeSeriesSplit(n_splits=cv)
    pbar = tqdm(tscv.split(X_numeric))
    for train_index, test_index in pbar:
        X_month_train, X_day_train, X_hour_train, X_numeric_train, y_train = X_month[train_index], X_day[train_index], X_hour[train_index], X_numeric[train_index], y[train_index]
        X_month_test, X_day_test, X_hour_test, X_numeric_test, y_test = X_month[test_index], X_day[test_index], X_hour[test_index], X_numeric[test_index], y[test_index]

        model = Model()
        train_model(model, X_month_train, X_day_train, X_hour_train, X_numeric_train, y_train, epochs=10)
        y_pred = model(X_month_test, X_day_test, X_hour_test, X_numeric_test)
        loss = nn.MSELoss()(y_pred, y_test)
        pbar.set_description(f"Fold Loss: {loss.item()}")

In [ ]:
# Load training dataset and perform cross-validation
X_month, X_day, X_hour, X_numeric, y = load_dataset('data/train.csv', 5)
cross_validate(X_month, X_day, X_hour, X_numeric, y, cv=5)